In [14]:
%reload_ext watermark
%matplotlib inline

import os
from metapool.metapool import *
from metapool import validate_and_scrub_sample_sheet, make_sample_sheet, IGMManifest
%watermark -i -v -iv -m -h -p metapool,sample_sheet,openpyxl -u

Last updated: 2023-03-29T10:53:57.920457-07:00

Python implementation: CPython
Python version       : 3.9.16
IPython version      : 8.11.0

metapool    : 0+untagged.134.gf573aae
sample_sheet: 0.13.0
openpyxl    : 3.1.2

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 21.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 16
Architecture: 64bit

Hostname: Applejack.lan

re        : 2.2.1
seaborn   : 0.12.2
pandas    : 1.5.3
matplotlib: 3.7.1
json      : 2.0.9
numpy     : 1.24.2



# Knight Lab metagenomics sample sheet generator 

### What is it?

This Jupyter Notebook allows you to automatically generate sample sheets for metagenomic sequencing. 


### Here's how it should work.

You'll start out with a **basic plate map** (platemap.tsv) , which just links each sample to it's approprite row and column.

You can use this google sheet template to generate your plate map:

https://docs.google.com/spreadsheets/d/1xPjB6iR3brGeG4bm2un4ISSsTDxFw5yME09bKqz0XNk/edit?usp=sharing

Next you'll automatically assign dual indexes to each sample in order to produce a **sample sheet** (samplesheet.csv) that you can give directly to IGM for sequencing. 

## Step 1: read in plate map

**Enter the correct path to the plate map file**. This will serve as the plate map for relating all subsequent information.

In [15]:
plate_map_fp = './test_data/Plate_Maps/Finrisk 33-36_plate_map.tsv'

if not os.path.isfile(plate_map_fp):
    print("Problem! %s is not a path to a valid file" % file)

**Read in the plate map**. It should look something like this:

```
Sample	Row	Col	Blank
GLY_01_012	A	1	False
GLY_14_034	B	1	False
GLY_11_007	C	1	False
GLY_28_018	D	1	False
GLY_25_003	E	1	False
GLY_06_106	F	1	False
GLY_07_011	G	1	False
GLY_18_043	H	1	False
GLY_28_004	I	1	False
```

**Make sure there a no duplicate IDs**. If each sample doesn't have a different name, an **error** will be thrown and you won't be able to generate a sample sheet.

In [16]:
plate_df = read_plate_map_csv(open(plate_map_fp,'r'))

plate_df.head()

/Users/ccowart/Development/Pipeline/metagenomics_pooling_notebook/metapool/metapool.py:214: UserWarning: No qiita_oauth2_conf_fp set so not checking plate_map and Qiita study overlap
  warnings.warn('No qiita_oauth2_conf_fp set so not checking plate_map '


,Sample,Row,Col,Blank,Project Plate,Project Name,Compressed Plate Name,Well
0,33-A1,A,1,True,FinRisk_P33,FinRisk_10855,FinRisk Plate 33-36,A1
1,820072905-2,C,1,False,FinRisk_P33,FinRisk_10855,FinRisk Plate 33-36,C1
2,820029517-3,E,1,False,FinRisk_P33,FinRisk_10855,FinRisk Plate 33-36,E1
3,820073753-4,G,1,False,FinRisk_P33,FinRisk_10855,FinRisk Plate 33-36,G1
4,820049719-1,I,1,False,FinRisk_P33,FinRisk_10855,FinRisk Plate 33-36,I1


# Assign barcode combination

This portion of the notebook will assign a unique combination of dual indeces to each sample. 

As inputs, it requires:
1. A plate map dataframe (from previous step)
2. A tab-delimited index combination file, relating index combinations, i5 and i7 index values, and i5 and i7 index locations

The workflow then:
1. reads in the index combo list
2. assigns indices per sample

## Step 2: Read in index combo list

This is a file that contains every possible i5 and i7 barcode combo on a separate line,
along with plate and well location information. It should look something like this:

```
index combo,index combo seq,i5 name,i5 sequence,i5 well,i5 plate,i7 name,i7 sequence,i7 well,i7 plate
0,ACCGACAAACGTTACC,iTru5_01_A,ACCGACAA,A1,iTru5_plate,iTru7_101_01,ACGTTACC,A1,iTru7_plate
1,AGTGGCAACTGTGTTG,iTru5_01_B,AGTGGCAA,B1,iTru5_plate,iTru7_101_02,CTGTGTTG,A2,iTru7_plate
2,CACAGACTTGAGGTGT,iTru5_01_C,CACAGACT,C1,iTru5_plate,iTru7_101_03,TGAGGTGT,A3,iTru7_plate
3,CGACACTTGATCCATG,iTru5_01_D,CGACACTT,D1,iTru5_plate,iTru7_101_04,GATCCATG,A4,iTru7_plate
4,GACTTGTGGCCTATCA,iTru5_01_E,GACTTGTG,E1,iTru5_plate,iTru7_101_05,GCCTATCA,A5,iTru7_plate
5,GTGAGACTAACAACCG,iTru5_01_F,GTGAGACT,F1,iTru5_plate,iTru7_101_06,AACAACCG,A6,iTru7_plate
6,GTTCCATGACTCGTTG,iTru5_01_G,GTTCCATG,G1,iTru5_plate,iTru7_101_07,ACTCGTTG,A7,iTru7_plate
7,TAGCTGAGCCTATGGT,iTru5_01_H,TAGCTGAG,H1,iTru5_plate,iTru7_101_08,CCTATGGT,A8,iTru7_plate
8,CTTCGCAATGTACACC,iTru5_02_A,CTTCGCAA,I1,iTru5_plate,iTru7_101_09,TGTACACC,A9,iTru7_plate
```

In [17]:
index_combo_fp = './test_output/iTru/temp_iTru_combos.csv'

if not os.path.isfile(index_combo_fp):
    print("Problem! %s is not a path to a valid file" % file)

In [18]:
index_combos = pd.read_csv(index_combo_fp)
index_combos.head()

,index combo,index combo seq,i5 name,i5 sequence,i5 well,i5 plate,i7 name,i7 sequence,i7 well,i7 plate
0,0,AACGCACACTCGTCTT,iTru5_19_A,AACGCACA,A1,iTru5_plate,iTru7_109_01,CTCGTCTT,A22,iTru7_plate
1,1,ATGCCTAGCGAACTGT,iTru5_19_B,ATGCCTAG,B1,iTru5_plate,iTru7_109_02,CGAACTGT,B22,iTru7_plate
2,2,CATACGGACATTCGGT,iTru5_19_C,CATACGGA,C1,iTru5_plate,iTru7_109_03,CATTCGGT,C22,iTru7_plate
3,3,GGTCACTATCGGTTAC,iTru5_19_D,GGTCACTA,D1,iTru5_plate,iTru7_109_04,TCGGTTAC,D22,iTru7_plate
4,4,GTATTCCGAAGTCGAG,iTru5_19_E,GTATTCCG,E1,iTru5_plate,iTru7_109_05,AAGTCGAG,E22,iTru7_plate


## Step 3: Assign index combo

This will pick a set of index combos from the index combo for the number of samples in the `plate_df` DataFrame.

Specify the PrimerPlate you used to assign barcodes. 

In [19]:
primerplate_combo = 1

starting_combo = (primerplate_combo - 1)*384

indices = assign_index(len(plate_df['Sample']), index_combos, start_idx=starting_combo).reset_index()

plate_df = pd.concat([plate_df, indices], axis=1)

plate_df.head()

,Sample,Row,Col,Blank,Project Plate,Project Name,Compressed Plate Name,Well,index,index combo,index combo seq,i5 name,i5 sequence,i5 well,i5 plate,i7 name,i7 sequence,i7 well,i7 plate
0,33-A1,A,1,True,FinRisk_P33,FinRisk_10855,FinRisk Plate 33-36,A1,0,0,AACGCACACTCGTCTT,iTru5_19_A,AACGCACA,A1,iTru5_plate,iTru7_109_01,CTCGTCTT,A22,iTru7_plate
1,820072905-2,C,1,False,FinRisk_P33,FinRisk_10855,FinRisk Plate 33-36,C1,1,1,ATGCCTAGCGAACTGT,iTru5_19_B,ATGCCTAG,B1,iTru5_plate,iTru7_109_02,CGAACTGT,B22,iTru7_plate
2,820029517-3,E,1,False,FinRisk_P33,FinRisk_10855,FinRisk Plate 33-36,E1,2,2,CATACGGACATTCGGT,iTru5_19_C,CATACGGA,C1,iTru5_plate,iTru7_109_03,CATTCGGT,C22,iTru7_plate
3,820073753-4,G,1,False,FinRisk_P33,FinRisk_10855,FinRisk Plate 33-36,G1,3,3,GGTCACTATCGGTTAC,iTru5_19_D,GGTCACTA,D1,iTru5_plate,iTru7_109_04,TCGGTTAC,D22,iTru7_plate
4,820049719-1,I,1,False,FinRisk_P33,FinRisk_10855,FinRisk Plate 33-36,I1,4,4,GTATTCCGAAGTCGAG,iTru5_19_E,GTATTCCG,E1,iTru5_plate,iTru7_109_05,AAGTCGAG,E22,iTru7_plate


# Make HiSeq sample sheet

This workflow takes the pooled sample information and writes an Illumina sample sheet that can be given directly to the sequencing center. 

As inputs, this notebook requires:
1. A plate map DataFrame (from previous step)

The workflow:
1. formats sample names as bcl2fastq-compatible
2. formats sample data
3. sets values for sample sheet fields and formats sample sheet.
4. writes the sample sheet to a file

## Step 1: Format sample names to be bcl2fastq-compatible

bcl2fastq requires *only* alphanumeric, hyphens, and underscore characters. We'll replace all non-those characters
with underscores and add the bcl2fastq-compatible names to the DataFrame.

In [20]:
plate_df['sample sheet Sample_ID'] = plate_df['Sample'].map(bcl_scrub_name)

plate_df.head()

,Sample,Row,Col,Blank,Project Plate,Project Name,Compressed Plate Name,Well,index,index combo,index combo seq,i5 name,i5 sequence,i5 well,i5 plate,i7 name,i7 sequence,i7 well,i7 plate,sample sheet Sample_ID
0,33-A1,A,1,True,FinRisk_P33,FinRisk_10855,FinRisk Plate 33-36,A1,0,0,AACGCACACTCGTCTT,iTru5_19_A,AACGCACA,A1,iTru5_plate,iTru7_109_01,CTCGTCTT,A22,iTru7_plate,33-A1
1,820072905-2,C,1,False,FinRisk_P33,FinRisk_10855,FinRisk Plate 33-36,C1,1,1,ATGCCTAGCGAACTGT,iTru5_19_B,ATGCCTAG,B1,iTru5_plate,iTru7_109_02,CGAACTGT,B22,iTru7_plate,820072905-2
2,820029517-3,E,1,False,FinRisk_P33,FinRisk_10855,FinRisk Plate 33-36,E1,2,2,CATACGGACATTCGGT,iTru5_19_C,CATACGGA,C1,iTru5_plate,iTru7_109_03,CATTCGGT,C22,iTru7_plate,820029517-3
3,820073753-4,G,1,False,FinRisk_P33,FinRisk_10855,FinRisk Plate 33-36,G1,3,3,GGTCACTATCGGTTAC,iTru5_19_D,GGTCACTA,D1,iTru5_plate,iTru7_109_04,TCGGTTAC,D22,iTru7_plate,820073753-4
4,820049719-1,I,1,False,FinRisk_P33,FinRisk_10855,FinRisk Plate 33-36,I1,4,4,GTATTCCGAAGTCGAG,iTru5_19_E,GTATTCCG,E1,iTru5_plate,iTru7_109_05,AAGTCGAG,E22,iTru7_plate,820049719-1


## Step 2: format sample sheet data

This step formats the data columns appropriately for the sample sheet, using the values we've calculated previously.

The newly-created bcl2fastq-compatible names will be in the **`Sample ID`** and **`Sample Name`** columns. The
original sample names will be in the **`Description`** column.

Modify **`lanes`** to indicate which lanes this pool will be sequenced on.

**Project Name and Project Plate values will be placed in the **`Sample_Project`** and **`Sample_Name`**
columns, respectively.

**`sequencer`** is important for making sure the i5 index is in the correct orientation for demultiplexing. `HiSeq4000`, `HiSeq3000`, `NextSeq`, and `MiniSeq` all require reverse-complemented i5 index sequences. If you enter one of these exact strings in for `sequencer`, it will revcomp the i5 sequence for you.

`HiSeq2500`, `MiSeq`, and `NovaSeq` will not revcomp the i5 sequence. 

In [21]:
plate_df['Project Name'] = 'Example_Project_1'
plate_df['Project Plate'] = 'Example Plate 1'
sequencer = 'HiSeq4000'
lanes = [1, 2, 3, 4]

# Knight Lab Nextera is also valid
metadata = {
    'Bioinformatics': [
        {
         'Sample_Project': 'Example_Project_1',
         'QiitaID': '1',
         'BarcodesAreRC': 'False',
         'ForwardAdapter': 'GATCGGAAGAGCACACGTCTGAACTCCAGTCAC',
         'ReverseAdapter': 'GATCGGAAGAGCGTCGTGTAGGGAAAGGAGTGT',
         'HumanFiltering': 'True',
         'library_construction_protocol': 'Knight Lab KAPA HyperPlus',
         'experiment_design_description': 'Example_Project_1_Description',
        },
    ],
    'Contact': [
        {
         'Sample_Project': 'Example_Project_1',
         # non-admin contacts who want to know when the sequences
         # are available in Qiita
         'Email': 'jonsan@gmail.com'
        },
    ],
    'Assay': 'Metagenomics',
}

sheet = make_sample_sheet(metadata, plate_df, sequencer, lanes)

HiSeq4000: i5 barcodes are output as reverse compliments


Validate the contents of the sample sheet

In [22]:
sheet = validate_and_scrub_sample_sheet(sheet)

## Step 4: Write the sample sheet to file

In [23]:
# write sample sheet as .csv
sample_sheet_fp = './test_output/SampleSheets/YYYY_MM_DD_FinRisk_33-36_samplesheet.csv'

if os.path.isfile(sample_sheet_fp):
    print("Warning! This file exists already.")

Warning! This file exists already.


In [24]:
with open(sample_sheet_fp,'w') as f:
    sheet.write(f)
    
!head -n 30 {sample_sheet_fp}
!echo ...
!tail -n 15 {sample_sheet_fp}

[Header],,,,,,,,,,
IEMFileVersion,4,,,,,,,,,
Investigator Name,Knight,,,,,,,,,
Experiment Name,RKL_experiment,,,,,,,,,
Date,2023-03-29,,,,,,,,,
Workflow,GenerateFASTQ,,,,,,,,,
Application,FASTQ Only,,,,,,,,,
Assay,Metagenomics,,,,,,,,,
Description,,,,,,,,,,
Chemistry,Default,,,,,,,,,
,,,,,,,,,,
[Reads],,,,,,,,,,
151,,,,,,,,,,
151,,,,,,,,,,
,,,,,,,,,,
[Settings],,,,,,,,,,
ReverseComplement,0,,,,,,,,,
MaskShortReads,1,,,,,,,,,
OverrideCycles,Y151;I8N2;I8N2;Y151,,,,,,,,,
,,,,,,,,,,
[Data],,,,,,,,,,
Sample_ID,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2,Sample_Project,Well_description,Lane
33-A1,33-A1,Example Plate 1,A1,iTru7_109_01,CTCGTCTT,iTru5_19_A,TGTGCGTT,Example_Project_1,33-A1,1
820072905-2,820072905-2,Example Plate 1,C1,iTru7_109_02,CGAACTGT,iTru5_19_B,CTAGGCAT,Example_Project_1,820072905-2,1
820029517-3,820029517-3,Example Plate 1,E1,iTru7_109_03,CATTCGGT,iTru5_19_C,TCCGTATG,Example_Project_1,820029517-3,1
820073753-4,820073753-4,Example Plate 1,G1,iT

# Create a Manifest file for IGM

Make sure you update the correct values for the attributes below. If you need to update the defaults, type `fest.` and hit tab to see what other values can be changed. The number of samples will be determined based on the number of samples in the sample sheet, but you have to enter the value for the number of lanes, and the names of the pools (one per line).

In [25]:
fest = IGMManifest()

fest.number_of_samples = len(sheet)
fest.number_of_lanes = 1

# remove plate naming
fest.pools = [
    'FILL IN POOL NAME',
    # 'FILL IN SECOND POOL NAME ...',
]

fest.write('./test_output/SampleSheets/YYYY_MM_DD_PI_Sequencing_Runs_Manifest_2021.xlsx')

Saving manifest to ./test_output/SampleSheets/YYYY_MM_DD_PI_Sequencing_Runs_Manifest_2021.xlsx
Date of Sample Submission: 03/29/23
Institute/Company Name: Knight Lab
PI Name: Dr. Knight
PI Email: mackenzie.m.bryant@gmail.com
Contact Name: MacKenzie Bryant
Contact Email: mackenzie.m.bryant@gmail.com
Project Number: 2004033
Task Number: 1
Platform: NovaSeq S4
Run Type: PE150
Custom Primer? (Provide more info in comments box): No-Standard Illumina Primers are fine
Total number of Samples: 1536
Total number of Lanes OR Total Reads Required: 1

Sample Name	Pool Name	Library Size (bp)	Library Prep Method
FILL IN POOL NAME	FILL IN POOL NAME	500	KHP
